In [3]:
import torch, torchvision
from PIL import Image
from urllib.request import urlopen

# load model
resnet = torchvision.models.resnet101(pretrained=True)

# set network to evaluation mode
resnet.eval()

transform = torchvision.transforms.Compose([          
 torchvision.transforms.Resize(256),                   
 torchvision.transforms.CenterCrop(224),               
 torchvision.transforms.ToTensor(),                     
 torchvision.transforms.Normalize(                      
 mean=[0.485, 0.456, 0.406],                            
 std=[0.229, 0.224, 0.225]                             
 )])

img = Image.open(urlopen("https://upload.wikimedia.org/wikipedia/en/5/5f/Original_Doge_meme.jpg"))
img_t = transform(img)
print(img_t.shape)

batch_t = torch.unsqueeze(img_t, 0)


# perform inference
out = resnet(batch_t)

# print top-5 classes predicted by model
_, indices = torch.sort(out, descending=True)
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
for idx in indices[0][:5]:
  print('Label:', idx, '. Confidence Score:', percentage[idx].item(), '%')


torch.Size([3, 224, 224])
Label: tensor(273) . Confidence Score: 81.02312469482422 %
Label: tensor(207) . Confidence Score: 2.8767271041870117 %
Label: tensor(248) . Confidence Score: 2.681793451309204 %
Label: tensor(263) . Confidence Score: 2.074437141418457 %
Label: tensor(235) . Confidence Score: 1.7300022840499878 %


In [4]:
!jupyter nbconvert --to script homework.ipynb

[NbConvertApp] WARNING | Config option `kernel_spec_manager_class` not recognized by `NbConvertApp`.
[NbConvertApp] Converting notebook homework.ipynb to script
[NbConvertApp] Writing 1098 bytes to homework.py
